### Prepare Data

In [47]:
import string
from random import shuffle
x = [[i] for i in range(10)]


FEMALE_FILE = 'Baby_names/female.txt';
MALE_FILE = 'Baby_names/male.txt';
N_ALPHABET = 28; #Add 2 extra for encoding the end of name EON and the dash '-'
MAX_CHARS = 16; #Assume no name is longer than 16 chars

alphabet_id = {}
for i in xrange(26):
    alphabet_id[string.lowercase[i]] = i;

alphabet_id['-'] = 26;
alphabet_id['\n'] = 27;

id2char = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',\
           'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', \
           's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-', '\n'];

def str2onehot(string):
    string = string.lower();
    result = np.zeros((MAX_CHARS, N_ALPHABET));
    
    chars = list(string);
    idx = [alphabet_id[char] for char in chars];
    result[range(len(string)), idx] = 1;
    return result;
input_male = []
target_male = []
sequence_len = []
with open(MALE_FILE) as fp:
    for line in fp:
        if (line[0]!="#" and line[0]!="\n"):
            input_male.append(str2onehot(line[:-1]))
            target_male.append(str2onehot(line[1:]))
            sequence_len.append(len(line)-1);
n = len(input_male);
idx = [i for i in range(n)]
shuffle(idx)
input_male = [input_male[i] for i in idx]
target_male = [target_male[i] for i in idx]
sequence_len = [sequence_len[i] for i in idx]

### Build the LSTM graph

In [48]:
import tensorflow as tf
tf.reset_default_graph()

max_chars = max([name.shape[0] for name in input_male]);
#Placeholder for input and target, [Batch Size, Sequence Length, Input Dimension]
data = tf.placeholder(tf.float32, [None, max_chars, N_ALPHABET])
target = tf.placeholder(tf.float32, [None, max_chars, N_ALPHABET])

num_hidden = 100
time_steps = max_chars

lstmCell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)

seqlen = tf.placeholder(tf.int32, [None])

#unroll the LSTM network
outputs, states = tf.nn.dynamic_rnn(cell=lstmCell, inputs=data, sequence_length=seqlen, dtype=tf.float32)

# Define weights
out_size = N_ALPHABET;

# Weights for each timestep output, using the same weight
weight = tf.Variable(tf.truncated_normal([num_hidden, out_size], stddev=0.1))
bias = tf.Variable(tf.constant(0.1, shape=[out_size]))

outputs = tf.reshape(outputs, [-1, num_hidden])
prediction = tf.nn.softmax(tf.matmul(outputs, weight) + bias)
prediction = tf.reshape(prediction, [-1, time_steps, out_size])

cross_entropy = -tf.reduce_sum(target * tf.log(prediction))
#cross_entropy = tf.reduce_mean(cross_entropy)

#cross_entropy = -tf.reduce_sum(target * tf.log(prediction))
final_prediction = tf.argmax(prediction, 2)
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = tf.not_equal(tf.argmax(target, 2), tf.argmax(prediction, 2))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

### Execute the graph

In [49]:
from numpy import random
def pred2name(pred):
    #ndarray of n_time_steps * N_ALPHABET
    name = [id2char[char_id] for char_id in pred];
    return "".join(name)

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

batch_size = 300
no_of_batches = int(len(input_male)/batch_size)
epoch = 200

for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out, seql = input_male[ptr:ptr+batch_size], target_male[ptr:ptr+batch_size], sequence_len[ptr:ptr+batch_size]   
        ptr+=batch_size
        result = sess.run([minimize, cross_entropy, final_prediction, error], {data: inp, target: out, seqlen: seql})
        print(result[1])
        #print(result[3])
        prediction = result[2];
        idx = random.choice(range(prediction.shape[0]), size=2);
        preds = prediction[idx]
        for pred in preds:
            sample_name = pred2name(pred)
            print repr(sample_name)
            
    print "Epoch - ",str(i)

sess.close();



5832.75
'u\nxaaaaaaaaaaaaa'
'zzzzyaaaaaaaaaaa'
5753.66
'beooaaaaaaaaaaaa'
'xb\nb\nbaaaaaaaaaa'
5951.13
'iazzllrrrrrrrrrr'
'\n\n\n\nrrrrrrrrrrrr'
5950.44
'\noe\nerrrrrrrrrrr'
'd\n\negrrrrrrrrrrr'
5851.98
'd\n\n\n\n\nrrrrrrrrrr'
'\n\n\n\n\n\nrrrrrrrrrr'
5790.91
'nn\n\nrrrrrrrrrrrr'
'\nden\n\nrrrrrrrrrr'
5866.88
'\n\n\n\nr\nrrrrrrrrrr'
'\n\n\n\nrrrrrrrrrrrr'
5891.03
'e\n\n\n\n\n\n\n\nrrrrrrr'
'\n\n\n\n\n\n\nrrrrrrrrr'
5877.7
'e\n\nrrrrrrrrrrrrr'
'i\n\n\n\n\n\nrrrrrrrrr'
Epoch -  0
5740.36
'n\n\n\n\n\n\n\nrrrrrrrr'
'\n\n\nrrrrrrrrrrrrr'
5660.59
'i\n\n\nrrrrrrrrrrrr'
'ia\n\n\n\n\n\nrrrrrrrr'
5847.82
'\n\n\n\n\n\n\n\nrrrrrrrr'
'ee\n\n\n\n\nrrrrrrrrr'
5839.4
'\n\n\n\n\n\nrrrrrrrrrr'
'nn\nn\n\n\nrrrrrrrrr'
5734.81
'n\ne\n\n\n\nrrrrrrrrr'
'ee\n\n\n\nrrrrrrrrrr'
5664.57
'e\n\n\n\n\n\n\nrrrrrrrr'
'\n\n\n\n\nrrrrrrrrrrr'
5722.78
'e\n\n\n\n\n\n\n\n\n\n\n\nrrr'
'\n\n\n\n\nrrrrrrrrrrr'
5733.57
'n\n\n\n\n\n\nrrrrrrrrr'
'\n\n\n\nrrrrrrrrrrrr'
5697.5
'e\n\n\n\n\nrrrrrrrrrr'
'e\n\n\n\n\n\n\nrrrrrrrr'
Epo

In [31]:
sess.close()

In [68]:
target_male[0]

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.]])

In [44]:
from random import shuffle
idx = [i for i in range(10)]
shuffle(idx)


In [45]:
idx

[4, 5, 7, 9, 8, 3, 6, 2, 1, 0]

In [43]:
type(x)

NoneType